In [1]:
import numpy as np
import pandas as pd
from  skimage.io import imread, imshow
from pathlib import Path
import pywt
import pywt.data
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

c:\Users\pm114\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
def get_feature(picture, cortes):
  LL = picture
  for i in range(cortes):
     LL, (LH, HL, HH) = pywt.dwt2(LL, 'haar')
  return LL.flatten().tolist()

In [3]:
path = "../imagenes_1/"
cv = []
entries = Path(path)
for entry in entries.iterdir():
  img = path + entry.name
  picture = imread(img)
  cv.append(get_feature(picture,2) + [entry.name[1:3]])

#print(cv[0][len(cv)-1])

In [30]:
# Normalize data
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(cv)
print(normalized_data)


[[0.51985294 0.53578431 0.48872549 ... 0.58455882 0.62107843 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [1.         1.         1.         ... 1.         1.         0.        ]
 ...
 [0.26470588 0.35490196 0.38137255 ... 0.30392157 0.30882353 1.        ]
 [0.37009804 0.48357843 0.45710784 ... 0.62941176 0.63823529 1.        ]
 [0.58529412 0.51127451 0.59583333 ... 0.26470588 0.27671569 1.        ]]


In [4]:
dataset = pd.DataFrame(cv)
#dataset.insert(0,'BIAS',1)

print(dataset)

       0        1        2        3        4        5        6        7      \
0     530.25   546.50   498.50   504.50   489.25   416.00   359.25   430.25   
1       0.00     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
2    1020.00  1020.00  1020.00  1018.00  1018.00  1018.00  1019.00  1020.00   
3    1020.00  1020.00  1020.00  1020.00  1020.00  1019.00  1019.00  1020.00   
4     215.00   306.25   279.75   334.75   233.00   255.75   306.75   282.00   
..       ...      ...      ...      ...      ...      ...      ...      ...   
827   204.00   204.00   204.00   204.00   204.00   204.00   204.00   204.00   
828   371.50   346.50   345.00   381.25   398.25   333.25   481.00   409.50   
829   270.00   362.00   389.00   404.00   397.00   351.00   301.75   475.00   
830   377.50   493.25   466.25   497.50   471.00   377.25   387.00   401.00   
831   597.00   521.50   607.75   527.50   457.00   476.25   477.75   499.00   

       8        9      ...    14991    14992    149



**Hipótesis**:

- Ecuación de la recta o Hiperplano
\begin{equation}
h(x_i) = w_0 + w_1x_i^1 +  w_2x_i^2 ... w_kx_i^k
\end{equation} \\

- Ecuación de la función sigmoidea (clasificador binario)
\begin{equation}
s(x_i) = \frac{1}{1 + e^{-h(x_i)}}
\end{equation}






In [32]:
def H(x,w):
  return np.dot(x,w.T)

def S(x,w):
  return 1 / (1 + np.exp(-H(x,w)))

- **Loss Function** (Cross-Entropy)

\begin{equation}
L = -\frac{1}{n}\sum_{i=0}^n(y_ilog(s(x_i)) + (1-y_i)log(1-s(x_i)))  
\end{equation} \\


In [33]:
def Loss_function(x,y,w):
  f1=y*np.log(S(x,w))
  f2=(1-y)*np.log(1-S(x,w))
  f3= np.sum(f1+f2)
  return -1/len(y)*(f3)

- **Derivatives**

\begin{equation}
\frac{\partial L}{\partial w_j} = \frac{1}{n}\sum_{i=0}^n(y_i - s(x_i))(-x_i^j)
\end{equation} \\

Nota:  $x_i^j$ se refiere a la característica $j-esima$ del objeto $i-esimo$ de entrenamiento


In [34]:
def D(x,y,w):
  n = len(x[0])
  num = len(w)
  gradient = [0.0] * num

  for i in range(num):
    sum_acum = 0.0
    for j in range(n):
      sum_acum += (y[j] - S(x[j],w)) * (-x[i][j])
    gradient[i] = (1.0/n) * sum_acum
  return gradient

#def D(x,y,w):
#  return np.matmul((y - S(x,w)),-x)/len(y)

- Change parameters

\begin{equation}
 w_j = w_i - \alpha\frac{\partial L}{\partial w_j}
\end{equation}

In [35]:
def change_parameters(w, derivatives, alpha):
  w_n = [0.0]*len(w)
  for j in range(len(w)):
    w_n[j] = w[j] - alpha * derivatives[j]
  return w_n

  #return w - (alpha * derivatives)

In [36]:
def training(x,y, epochs, alpha):
  print(len(x))
  w=np.ones(x.shape[1])
  Loss=[]
  for i in range(epochs):
    L =  Loss_function(x,y,w)
    dw = D(x,y,w)
    w =  change_parameters(w, dw, alpha)
    Loss.append(L)
  return w,Loss

In [37]:
def Testing(x_test, y_test,w):
   y_pred = S(x_test,w)
   y_pred = np.round(y_pred)
   correctos = np.sum(y_pred == y_test)
   print(f"Número de datos correctos:{correctos}")
   porc_aciertos= (correctos/len(y_test))*100
   print(f"Porcentaje de aciertos:{porc_aciertos}%")
   print(f"Porcentaje de error:{100-porc_aciertos}%")


In [38]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


NameError: name 'Y' is not defined

In [ ]:
# import matplotlib.pyplot as plt
# def plot_loss(loss):
#     plt.figure()
#     plt.plot(loss)
#     plt.title("loss function")
#     plt.xlabel("Epohs")
#     plt.ylabel("Loss")
#     plt.show()

# plot_loss(loss)
# Testing(x_test,y_test,w)